In [ ]:
from bs4 import BeautifulSoup
import requests
import pickle
from lxml import html
from Swimmer import *
import selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

## Inserire il codice dello stile che si desidera selezionare
### Stile Libero:
###### "00" 50mt, "01" 100mt, "02" 200mt, "04" 400mt, "08" 800mt, "15" 1500mt
### Dorso:
##### "20" 50mt, "21" 100mt, "22" 200mt
### Rana:
##### "30" 50mt, "31" 100mt, "32" 200mt
### Farfalla:
##### "40" 50 mt, "41" 100mt, "42" 200mt
### Misti:
##### "51" 100mt, "52" 200mt, "54" 400mt

In [ ]:
Stile = "02"

## Inserire lunghezza vascha

In [ ]:
Vasc = "50"

## Inserire sesso dei nuotatori

In [ ]:
Sex = "M"

In [ ]:
# Creaiamo una risorsa Browser per poter scaricare i dati
url = "http://online.federnuoto.it/graduatorie/mas_atleti.php"

driver = webdriver.Firefox()

driver.get(url)

In [ ]:
# Selezioniamo le Form e inseriamo i dati al loro interno
righe = driver.find_element_by_name("txtnurighe")
righe.clear()
righe.send_keys("999")
annoag = Select(driver.find_element_by_name("txtanno"))
annoag.select_by_value("ALLTIME")
annoin = driver.find_element_by_name("txtdallanno")
annoin.clear()
annoin.send_keys("1900")
annofin = driver.find_element_by_name("txtallanno")
annofin.clear()
annofin.send_keys("1967")
sesso = Select(driver.find_element_by_name("txtsesso"))
sesso.select_by_value(Sex)
vasca = Select(driver.find_element_by_name("txtvasca"))
vasca.select_by_value(Vasc)
gara = Select(driver.find_element_by_name("txtgara"))
gara.select_by_value(Stile)

In [ ]:
# Simuliamo l'inserimento dei dati e salviamo il risultato ottenuto
driver.find_element_by_name("xricerca").click()

content = driver.page_source

with open("nuotatori_"+Stile+".html", "w") as f:
    f.write(content)

In [ ]:
#apriamo il file html
with open("nuotatori_"+Stile+".html") as file:
    dirty_strings = file.read()

In [ ]:
dirty_strings = dirty_strings.split("onclick=\"window.open('")

#filtra i link per ogni nuotatore
dirty_strings = list(filter(lambda x: "mas_schedaris.php" == x[:len("mas_schedaris.php")],
                            dirty_strings))


In [ ]:
#crea dei link validi
clean_strings = [x.split("\'")[0] for x in dirty_strings]

final_strings = ["http://online.federnuoto.it/graduatorie/"+"&".join(x.split(";")) for x in clean_strings]

In [ ]:
#salva i link validi
with open("nuotatori_filtrati.html","w") as file:
    for link in final_strings:
        file.write(link+"\n")

In [ ]:
#funzione che dato un link ci ritorna il nome del nuotatore
def get_name(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    a = a.split("""color:#000080">""")
    return a[1].split("(")[0].strip()

In [ ]:
#funzione che dato un link ci ritorna l'anno di nascita del nuotatore
def get_class(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    a = a.split("""color:#000080">""")
    return a[1].split("(")[1][:4]

In [ ]:
#funzione che dato un link ci ritorna data e tempo di ogni gara del nuotatore
def get_times_date(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    first_split = a.split("""<td align="center" height="20" style=" font-family:Verdana, Arial, Helvetica, sans-serif; font-weight:normal; font-style:normal; font-size:10px; color:#000080" width="10%">""")
    s = """</td> <td align="left" height="20" style=" font-family:Verdana, Arial, Helvetica, sans-serif; font-weight:normal; font-style:normal; font-size:10px; color:#000080" width="40%">"""
    second_split=[]

    for i in first_split:
        if s in i:
            second_split.append((i[2:10],i[13+len(s):len(s)+23]))
    
    return second_split

In [ ]:
#funzione che dato un link ci ritorna luogo di ogni gara del nuotatore
def get_locality(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    first_split = a.split("""</td> <td align="left" height="20" style=" font-family:Verdana, Arial, Helvetica, sans-serif; font-weight:normal; font-style:normal; font-size:10px; color:#000080" width="40%">""")
    s="""<title>   Federazione Italiana Nuoto"""
    split=[]

    for i in first_split:
        c = 0
        while i[15+c]!= "-":
            c = c + 1
        if i[15:15+(c-1)]!=s:
            split.append(i[15:15+(c-1)])
        
    return split

In [ ]:
#creo una lista di nuotatori
swimmers = []
i = 1
for url in final_strings:
    s = Swimmer()
    s.url = url
    s.name = get_name(url)
    s.times_date = get_times_date(url)
    s.year = get_class(url)
    s.locality = get_locality(url)
    swimmers.append(s)
    print(str(i) + ".", end="")
    if i % 25 == 0:
        print()
    i += 1
     

In [ ]:
#calcolo il rapporto e gli anni di partecipazione
for i in swimmers:
    i.fill_partecipation()
    i.calc_ratio(starting_year=Year(2000, 2001), ending_year=Year(2017, 2018))
    print(i)

In [ ]:
#salva nel database i nuotatori con pickle
with open("nuotatori_"+Stile+"_db.pkl","wb") as pkl:
    pickle.dump(swimmers, pkl)